In [1]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pymysql
import pandas as pd
from datetime import datetime, timedelta, date
import ast

In [4]:
from helpers import *

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [5]:
cursor = get_cursor()

In [6]:
from KalshiClientsBaseV2 import *

In [7]:
prod_email, prod_password = get_kalshi_creds()
prod_api_base = "https://trading-api.kalshi.com/trade-api/v2" 
exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

In [ ]:
spx_range_event_ticker, ndx_range_event_ticker = get_range_event_tickers(date(2024, 2, 23))
spx_ab_event_ticker, ndx_ab_event_ticker = get_above_below_event_tickers(date(2024, 2, 23))

ndx_ab_event = exchange_client.get_event(ndx_ab_event_ticker)
spx_ab_event = exchange_client.get_event(spx_ab_event_ticker)
ndx_range_event = exchange_client.get_event(ndx_range_event_ticker)
spx_range_event = exchange_client.get_event(spx_range_event_ticker)

ndx_ab_markets = [x['ticker'] for x in ndx_ab_event['markets']]
ndx_val_ab_ticker_map = map_value_to_ab_ticker(ndx_ab_markets) #maps lower bound to corresponding above/beloww ticker for NASDAQ100
spx_ab_markets = [x['ticker'] for x in spx_ab_event['markets']]
spx_val_ab_ticker_map = map_value_to_ab_ticker(spx_ab_markets) #maps lower bound to corresponding above/below ticker for SP500

ndx_range_markets = [x['ticker'] for x in ndx_range_event['markets']]
spx_range_markets = [x['ticker'] for x in spx_range_event['markets']]

ndx_range_subtitles = [(x['ticker'], x['subtitle']) for x in ndx_range_event['markets']]
spx_range_subtitles = [(x['ticker'], x['subtitle']) for x in spx_range_event['markets']]

ndx_range_ticker_to_ab_tickers = map_range_ticker_to_ab_tickers(ndx_range_subtitles, ndx_val_ab_ticker_map)
spx_range_ticker_to_ab_tickers = map_range_ticker_to_ab_tickers(spx_range_subtitles, spx_val_ab_ticker_map)
#{}_range_ticker_to_ab_tickers maps a range ticker to the 2 above/below tickers that can be used to replicate the payoff of the range ticker

markets = ndx_range_markets + ndx_ab_markets + spx_range_markets + spx_ab_markets

d = {'ticker': [], 'bid': [], 'ask': [], 'ts': []}
for ticker in markets:
    resp = exchange_client.get_market_history(ticker)
    cursor = resp['cursor']
    hist = resp['history']
    for h in hist:
        d['ticker'].append(ticker)
        d['bid'].append(h['yes_bid'])
        d['ask'].append(h['yes_ask'])
        d['ts'].append(datetime.fromtimestamp(h['ts']))
    while cursor is not None and len(cursor) > 0:
        resp = exchange_client.get_market_history(ticker, cursor = cursor)
        cursor = resp['cursor']
        hist = resp['history']
        for h in hist:
            d['ticker'].append(ticker)
            d['bid'].append(h['yes_bid'])
            d['ask'].append(h['yes_ask'])
            d['ts'].append(datetime.fromtimestamp(h['ts']))

df = pd.DataFrame(data=d)

arb_dict = {'range_ticker': [], 'fees': [], 'arb_edge': [],'per_contract_prof': [], 'ts': []}

for ndx_range_ticker in ndx_range_markets:
    try:
        lb_ticker = ndx_range_ticker_to_ab_tickers[ndx_range_ticker][0]
        ub_ticker = ndx_range_ticker_to_ab_tickers[ndx_range_ticker][1]
    except:
        continue
    
    rt_bid = 0
    lb_bid = 0
    ub_bid = 0
    last_ts = None
    rt_ask = 100
    lb_ask = 100
    ub_ask = 100
    
    temp_df = pd.concat([df[df['ticker']==ndx_range_ticker], df[df['ticker']==lb_ticker], df[df['ticker']==ub_ticker]], ignore_index=True)
    temp_df = temp_df.reset_index(drop=True).sort_values(by=['ts'])
    
    for idx, row in temp_df.iterrows():
        if row['ticker'] == ndx_range_ticker:
            rt_bid = row['bid']
            rt_ask = row['ask']
        elif row['ticker'] == lb_ticker:
            lb_bid = row['bid']
            lb_ask = row['ask']
        else:
            ub_bid = row['bid']
            ub_ask = row['ask']
        last_ts = row['ts']
        short_rt_fees = math.ceil(100 * (.035 * (rt_bid/100) * (1-rt_bid/100))) + math.ceil(100 * (.035 * (lb_ask/100) * (1-lb_ask/100))) + math.ceil(100 * (.035 * (ub_bid/100) * (1-ub_bid/100)))
        long_rt_fees = math.ceil(100 * (.035 * (rt_ask/100) * (1-rt_ask/100))) + math.ceil(100 * (.035 * (lb_bid/100) * (1-lb_bid/100))) + math.ceil(100 * (.035 * (ub_ask/100) * (1-ub_ask/100)))
        
        if rt_bid > lb_ask - ub_bid + short_rt_fees:
            arb_dict['range_ticker'].append(ndx_range_ticker)
            arb_dict['fees'].append(short_rt_fees)
            arb_dict['arb_edge'].append(rt_bid - lb_ask + ub_bid)
            arb_dict['per_contract_prof'].append(rt_bid - (lb_ask - ub_bid + short_rt_fees))
            arb_dict['ts'].append(last_ts)  
        elif rt_ask + long_rt_fees < lb_bid - ub_ask:
            arb_dict['range_ticker'].append(ndx_range_ticker)
            arb_dict['fees'].append(long_rt_fees)
            arb_dict['arb_edge'].append(lb_bid - ub_ask - rt_ask)
            arb_dict['per_contract_prof'].append((lb_bid - ub_ask) - rt_ask - long_rt_fees)
            arb_dict['ts'].append(last_ts)  

for spx_range_ticker in spx_range_markets:
    try:
        lb_ticker = spx_range_ticker_to_ab_tickers[spx_range_ticker][0]
        ub_ticker = spx_range_ticker_to_ab_tickers[spx_range_ticker][1]
    except:
        continue

    rt_bid = 0
    lb_bid = 0
    ub_bid = 0
    last_ts = None
    rt_ask = 100
    lb_ask = 100
    ub_ask = 100
    
    temp_df = pd.concat([df[df['ticker']==spx_range_ticker], df[df['ticker']==lb_ticker], df[df['ticker']==ub_ticker]], ignore_index=True)
    temp_df = temp_df.reset_index(drop=True).sort_values(by=['ts'])
    
    for idx, row in temp_df.iterrows():
        if row['ticker'] == spx_range_ticker:
            rt_bid = row['bid']
            rt_ask = row['ask']
        elif row['ticker'] == lb_ticker:
            lb_bid = row['bid']
            lb_ask = row['ask']
        else:
            ub_bid = row['bid']
            ub_ask = row['ask']
        last_ts = row['ts']
        short_rt_fees = math.ceil(100 * (.035 * (rt_bid/100) * (1-rt_bid/100))) + math.ceil(100 * (.035 * (lb_ask/100) * (1-lb_ask/100))) + math.ceil(100 * (.035 * (ub_bid/100) * (1-ub_bid/100)))
        long_rt_fees = math.ceil(100 * (.035 * (rt_ask/100) * (1-rt_ask/100))) + math.ceil(100 * (.035 * (lb_bid/100) * (1-lb_bid/100))) + math.ceil(100 * (.035 * (ub_ask/100) * (1-ub_ask/100)))

        if rt_bid > lb_ask - ub_bid + short_rt_fees:
            arb_dict['range_ticker'].append(spx_range_ticker)
            arb_dict['fees'].append(short_rt_fees)
            arb_dict['arb_edge'].append(rt_bid - lb_ask + ub_bid)
            arb_dict['per_contract_prof'].append(rt_bid - (lb_ask - ub_bid + short_rt_fees))
            arb_dict['ts'].append(last_ts)

        elif rt_ask + long_rt_fees < lb_bid - ub_ask:
            arb_dict['range_ticker'].append(spx_range_ticker)
            arb_dict['fees'].append(long_rt_fees)
            arb_dict['arb_edge'].append(lb_bid - ub_ask - rt_ask)
            arb_dict['per_contract_prof'].append((lb_bid - ub_ask) - rt_ask - long_rt_fees)
            arb_dict['ts'].append(last_ts)
arb = pd.DataFrame(data=arb_dict)

In [47]:
arb['per_contract_prof'].describe()['mean'] * arb['per_contract_prof'].describe()['count']

267.0